<a href="https://colab.research.google.com/github/yeesem/Deep_Learning/blob/main/Data_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Download the dataset
!wget https://storage.googleapis.com/tensorflow-1-public/course2/cats_and_dogs_filtered.zip

In [ ]:
import os
import zipfile

# Extract the archive
zip_ref = zipfile.ZipFile('./cats_and_dogs_filtered.zip','r')
zip_ref.extractall('tmp/')
zip_ref.close()

# Assign training and validation set directories
base_dir = 'tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir,'train')
validation_dir = os.path.join(base_dir,'validation')

# Directory with training cats pictures
train_cats_dir = os.path.join(train_dir,'cats')

# Directory with training dogs pictures
train_dogs_dir = os.path.join(train_dir,'dogs')

# Directory with validation cats pictures
validation_cats_dir = os.path.join(validation_dir,'cats')

# Directory with validation dogs pictures
validation_dogs_dir = os.path.join(validation_dir,'dogs')

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop

def create_model():
  # Creates CNN with 4 convolutional layers
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(32,(3,3),activation = 'relu',input_shape = (150,150,3)),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(64,(3,3),activation = 'relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(128,(3,3),activation = 'relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(128,(3,3),activation = 'relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(512,activation = 'relu'),
      tf.keras.layers.Dense(1,activation = 'sigmoid')
  ])

  model.compile(loss = 'binary_crossentropy',
                optimizer = RMSprop(learning_rate = 1e-4),
                metrics = ['accuracy'])

  return model


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def DataGenerator(DataAugmentation = False):

  if DataAugmentation == False:
    train_datagen = ImageDataGenerator(rescale = 1./255)

  else:
    train_datagen = ImageDataGenerator(
        rescale = 1./255,
        rotation_range = 40,
        width_shift_range = 0.2,
        height_shift_range = 0.2,
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip = True,
        fill_mode = 'nearest'
    )

  test_datagen = ImageDataGenerator(rescale = 1./255)

  train_generator = train_datagen.flow_from_directory(
      train_dir,
      target_size = (150,150),
      batch_size = 20,
      class_mode = 'binary'
  )

  validation_generator = test_datagen.flow_from_directory(
      validation_dir,
      target_size= (150,150),
      batch_size = 20,
      class_mode = 'binary'
  )

  return train_generator,validation_generator

In [1]:
def train_model(epoch,data_augmentation):
    # Create a new model
    model = create_model()
    train_generator,validation_generator = DataGenerator(data_augmentation)
    # Train the model
    history = model.fit(
        train_generator,
        steps_per_epoch = 100, # 2000 images = batch_size * steps
        epochs = epochs,
        validation_data = validation_generator,
        validation_steps = 50, # 1000 images = batch_size * steps
        verbose = 2
    )

    return history

In [ ]:
import matplotlib.pyplot as plt

def plot_loss_acc(history):
  # Plot the training and validation loss and accuracy
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(len(acc))

  # Plot the training and validation accuracy
  plt.plot(epochs,acc,'bo',label = 'Training accuracy')
  plt.plot(epochs,val_acc,'b',label = 'Validation accuracy')
  plt.title('Training and validation accuracy')
  plt.figure()

  # Plot the training and validation loss
  plt.plot(epochs,acc,'bo',label = 'Training accuracy')
  plt.plot(epochs,val_acc,'b',label = 'Validation accuracy')
  plt.title('Training and validation accuracy')
  plt.legend()
  plt.show()

In [ ]:
# Train the model without data augmentation
# Plot the training results
history = train_model(20,False)
plot_loss_acc(history)

In [ ]:
# Train the model with data augmentation
# Plot the training results
history = train_model(20,False)
plot_loss_acc(history)